# PNAS paper submission analysis

Analysing the wind characteristics of streaked shearwaters as they approach foraging points allows us to examine what conditions the birds travel in throughout foraging trips. 

## Previous study

In the most similar previous study ([Nevitt et al., 2008](https://www.pnas.org/content/105/12/4576)) investigating similar phenomena, the tracks of albatross approaching foraging points were examined and classified by their characteristics (in 4 groups: Direct, Turn, Zigzag, and Circle):

<p align="center">
   <img src=https://www.pnas.org/cms/10.1073/pnas.0709047105/asset/eb810a4f-38f4-4a23-beee-eb585774c043/assets/graphic/zpq0080896790001.jpeg alt="Albatross tracks approaching foraging" width="400">
</p>

And the histograms of relative wind bearings were examined for those categories:

 <p align="center">
    <img src=https://www.pnas.org/cms/10.1073/pnas.0709047105/asset/7d4e5b13-0f76-4fc1-b230-f77569c6a300/assets/graphic/zpq0080896790002.jpeg alt="Relative wind histograms" width="400">
 </p>
<!-- 
<p align="center>
   <img src= alt="Relative wind histograms">
</p> -->

This study used recordings of wandering albatross with GPS and stomach temperature transmitters. Foraging points were estimates as periods with rapid drops in stomach temperature, and surface landings taken from times when birds moved at under 2.8 m/s. Wind directions and speeds from all touchdown points were estimated using QuikSCAT daily level 3 gridded ocean wind vectors. This data comes from the SeaWinds scatterometer and records wind vector fields twice daily. Wind values cannot be produced for locations within 30km of land/ice. Backscatter recorded by the satellite is then processed through a Ku-band geophysical model function which derives surface wind speeds and direction. Rain contamination can be an issue due to difficulty in separating the backscatter effect of wind vs rain. The level value refers to the processing level of the data, starting at 0 (raw) through to 3 or 4.

In [1]:
using DataFrames, CSV, RCall, Plots, Geodesy, Dates, Distances, Statistics, Glob

┌ Warning: RCall.jl: During startup - Warning messages:
│ 1: Setting LC_COLLATE failed, using "C" 
│ 2: Setting LC_TIME failed, using "C" 
│ 3: Setting LC_MESSAGES failed, using "C" 
│ 4: Setting LC_MONETARY failed, using "C" 
└ @ RCall /Users/aran/.julia/packages/RCall/6kphM/src/io.jl:172


In [286]:
# FUNCTION FOR READING IN FORAGING AND WIND DATASETS
function readDat(dataLocation, pattern, IDpattern, colnames, header, years, DateFormats)
    files = glob(pattern, dataLocation)
    yrIDs = unique(getindex.(match.(r"(\d+)Shearwater.*",files),1) .* "_" .* getindex.(match.(IDpattern,files),1))
    ret = [DataFrame() for _ in 1:length(yrIDs)]
    for tg in 1:length(ret)
        tgFiles = files[occursin.(yrIDs[tg][1:4],files) .& occursin.("/"*yrIDs[tg][6:end],files)]
        for file in tgFiles
            append!(ret[tg], hcat(CSV.read(file, DataFrame, header = header), repeat([yrIDs[tg]], nrow(CSV.read(file, DataFrame, header = header)))), cols = :union)
        end
        rename!(ret[tg], colnames)
        # assign datetime
        ret[tg].DT = DateTime.(ret[tg].DT, DateFormats[occursin.(yrIDs[tg][1:4],years)])
    end
    return ret
end
# file locations for foraging and wind estimates
if Sys.iswindows()
    dataloc = "E:/My Drive/PhD/Data/"
else
    dataloc = "/Volumes/GoogleDrive-112399531131798335686/My Drive/PhD/Data/"
end
# bring in FORAGING AND GPS DATA
fDat = readDat(dataloc,"*/*/*/*/PredictedForage/*ForageGPS.txt",r".*PredictedForage/(.*)-20.*",[:DT,:lat,:lon,:forage,:yrID],1,["2018","2019"],[dateformat"d/m/y H:M:S.s",dateformat"d-u-y H:M:S.s"])

# bring in WIND ESTIMATES
wDat = readDat(dataloc, "*/*/MinDat/*.csv", r".*MinDat/(.*).csv", [:DT,:lat,:lon,:head,:X,:Y,:yrID], 0, ["2018","2019"],[dateformat"y-m-d H:M:S",dateformat"y-m-d H:M:S"]);

In [380]:
# add distance (m) and speed (kph) values
function dist(lat,lon)
    [sqrt.(diff(DataFrame(map(UTMZfromLLA(wgs84),LLA.(lat,lon))).x).^2 + diff(DataFrame(map(UTMZfromLLA(wgs84),LLA.(lat,lon))).y).^2);NaN]
end
function speed(dt,lat,lon)
    tdiff = Dates.value.(Second.(diff(dt)))
    spTrav = (dist(lat,lon)[1:end-1]./tdiff).*(3600/1000)
    return [spTrav;NaN]
end
for x in fDat
    x.distTrav = dist(x.lat,x.lon)
    x.spTrav = speed(x.DT, x.lat, x.lon)
end

# find the nearest time (index)
function findNearest(dt, time)
    argmin(abs.(dt .- time))
end
# calculate linearity
function linearity(dt,lat,lon,distance,twindow)
    out = repeat([NaN], length(dt))
    Array{Float64}(undef,findNearest(dt,(dt[end] - Minute(twindow))),1)
    for b = 1:findNearest(dt,(dt[end] - Minute(twindow)))
        nextPoint = b + findNearest(dt, dt[b] + (Minute(twindow))) - 1
        if abs(dt[nextPoint] - dt[b]) < Minute(5)
            out[b] = dist([lat[b],lat[nextPoint]],[lon[b],lon[nextPoint]])/sum(distance[b:nextPoint])
        else
            out[b] = NaN
        end
    end
end

linearity(fDat[1].DT,fDat[1].lat,fDat[1].lon,fDat[1].distTrav,51)

MethodError: MethodError: Cannot `convert` an object of type Vector{Float64} to an object of type Float64
Closest candidates are:
  convert(::Type{T}, !Matched::Base.TwicePrecision) where T<:Number at /usr/local/Cellar/julia/1.7.1_2/share/julia/base/twiceprecision.jl:262
  convert(::Type{T}, !Matched::AbstractChar) where T<:Number at /usr/local/Cellar/julia/1.7.1_2/share/julia/base/char.jl:185
  convert(::Type{T}, !Matched::CartesianIndex{1}) where T<:Number at /usr/local/Cellar/julia/1.7.1_2/share/julia/base/multidimensional.jl:136
  ...

87252

In [344]:
argmin(abs.(fDat[1].DT .- (fDat[1].DT[514] + Minute(3))))

550